In [1]:
!wandb login

wandb: Currently logged in as: dantle4564 (dantle4564-uc-santa-barbara) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
import os
filename = "/home/pranjali_jain/NCS/sparse-matrices/SPEC/SPEC189_2config/1-64-12/TRAIN/MISS"
os.listdir(filename)

['444.namd-120B',
 '458.sjeng-31B',
 '454.calculix-104B',
 '605.mcf_s-484B',
 '482.sphinx3-1522B',
 '459.GemsFDTD-1211B',
 '481.wrf-196B',
 '641.leela_s-334B',
 '473.astar-42B',
 '401.bzip2-7B',
 '453.povray-576B',
 '649.fotonik3d_s-1B',
 '623.xalancbmk_s-592B',
 '410.bwaves-1963B',
 '625.x264_s-18B',
 '482.sphinx3-234B',
 '623.xalancbmk_s-700B',
 '648.exchange2_s-72B',
 '641.leela_s-602B',
 '625.x264_s-39B',
 '459.GemsFDTD-1418B',
 '654.roms_s-842B',
 '482.sphinx3-1100B',
 '644.nab_s-5853B',
 '649.fotonik3d_s-8225B',
 '605.mcf_s-665B',
 '401.bzip2-226B',
 '429.mcf-217B',
 '641.leela_s-1083B',
 '482.sphinx3-1395B',
 '483.xalancbmk-127B',
 '481.wrf-455B',
 '603.bwaves_s-5359B',
 '444.namd-321B',
 '644.nab_s-12521B',
 '436.cactusADM-1804B',
 '619.lbm_s-3766B',
 '473.astar-153B',
 '456.hmmer-191B',
 '435.gromacs-134B',
 '648.exchange2_s-1712B',
 '654.roms_s-294B',
 '605.mcf_s-782B',
 '447.dealII-3B',
 '437.leslie3d-149B',
 '605.mcf_s-1644B',
 '462.libquantum-714B',
 '435.gromacs-226B',
 '

# IMPORTS

In [3]:
import random
from torchvision.utils import make_grid, save_image
from diffusers import DDPMScheduler
import joblib
import scipy.sparse as sp
from PIL import Image
import matplotlib.pyplot as plt
import wandb
from dataclasses import dataclass
import numpy as np
import math, os, glob, torch
from torch.utils.data import Dataset, DataLoader
from scipy import sparse
import torch.nn.functional as F
from pathlib import Path
from diffusers import DDPMScheduler

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/dantle/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Wandb Test

In [4]:
# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="dantle4564-uc-santa-barbara",
    # Set the wandb project where this run will be logged.
    project="my-awesome-project",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
)

# Simulate training.
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset

    # Log metrics to wandb.
    run.log({"acc": acc, "loss": loss})

# Finish the run and upload any remaining data.
run.finish()

wandb: Currently logged in as: dantle4564 (dantle4564-uc-santa-barbara) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


acc,▁▆▅█▇▇██
loss,█▆▂▂▁▁▁▁
acc,0.94238
loss,0.12036


# DOUBLE CHECK IN CASE HEATMAPS WERE NOT ADDED FOR SOME REASON

In [5]:
import os
import re
from natsort import natsorted

root = Path("FULL_DATASET")
train_full_benchmarks = os.listdir(str(root / "TRAIN" / "FULL"))
train_miss_benchmarks = os.listdir(str(root / "TRAIN" / "MISS"))
print(train_full_benchmarks)
print(train_miss_benchmarks)

print("FULL")
for folder in train_full_benchmarks:
    items = natsorted(os.listdir(str(root / "TRAIN" / "FULL" / folder)))
    print(folder, len(items), items[-1])

print("MISS")
for folder in train_miss_benchmarks:
    items = natsorted(os.listdir(str(root / "TRAIN" / "MISS" / folder)))
    print(folder, len(items), items[-1])


['401.bzip2-7B', '410.bwaves-1963B', '401.bzip2-226B', '429.mcf-217B', '435.gromacs-134B', '435.gromacs-226B', '429.mcf-51B', '429.mcf-192B', '410.bwaves-945B', '433.milc-337B', '410.bwaves-2097B', '435.gromacs-111B', '434.zeusmp-10B', '416.gamess-875B', '433.milc-274B', '429.mcf-184B', '401.bzip2-38B', '429.mcf-22B', '433.milc-127B']
['401.bzip2-7B', '410.bwaves-1963B', '401.bzip2-226B', '410.bwaves-945B', '410.bwaves-2097B', '416.gamess-875B', '401.bzip2-277B', '401.bzip2-38B']
FULL
401.bzip2-7B 2055 .DS_Store
410.bwaves-1963B 10075 410.bwaves-1963B.champsimtrace.xz.txt_12225_A.npz
401.bzip2-226B 7978 401.bzip2-226B.champsimtrace.xz.txt_7977_A.npz
429.mcf-217B 9852 429.mcf-217B.champsimtrace.xz.txt_9851_A.npz
435.gromacs-134B 7004 435.gromacs-134B.champsimtrace.xz.txt_7003_A.npz
435.gromacs-226B 7565 435.gromacs-226B.champsimtrace.xz.txt_7564_A.npz
429.mcf-51B 7453 429.mcf-51B.champsimtrace.xz.txt_7452_A.npz
429.mcf-192B 5811 429.mcf-192B.champsimtrace.xz.txt_5810_A.npz
410.bwaves-94

In [6]:
@dataclass
class TrainingConfig:
    image_size = 512 # the generated image resolution
    train_batch_size = 1
    eval_batch_size = 1  # how many images to sample during evaluation
    num_epochs = 1
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "diffusionTest"  # the model name locally and on the HF Hub

config = TrainingConfig()

In [7]:
from diffusers import UNet2DModel

model = UNet2DModel(
    sample_size=config.image_size,  # the target image resolution
    in_channels=3,  # the number of input channels, 3 for RGB images
    out_channels=3,  # the number of output channels
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channels for each UNet block
    down_block_types=(
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",  # a regular ResNet upsampling block
        "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
)

In [8]:
run = wandb.init(project="CacheBoxDModel", entity="dantle4564-uc-santa-barbara")

### Dataset Loader

In [9]:
def vector_to_square(x1d):
    L = x1d.shape[0]
    s = int(math.ceil(math.sqrt(L)))
    out = np.zeros(s*s, dtype=x1d.dtype)
    out[:L] = x1d
    return out.reshape(s, s)

class HeatmapSmallDataset(Dataset):
    def __init__(self, root, image_size=128):
        self.root = Path(root)
        self.image_size = int(image_size)
        # only load 20 benchmarks
        train_full_benchmarks = os.listdir(self.root / "TRAIN" / "FULL")[:21]
        train_miss_benchmarks = os.listdir(self.root / "TRAIN" / "MISS")[:21]
        full_files, miss_files = [],[]
        for bench in train_full_benchmarks:
            full_files += sorted(glob.glob(str(self.root / "TRAIN" / "FULL" / bench / "*_A.npz")))
        for bench in train_miss_benchmarks:
            miss_files += sorted(glob.glob(str(self.root / "TRAIN" / "MISS" / bench / "*_B.npz")))
        # full_files = [p for p in full_files if p.endswith("_A.npz")]
        # miss_files = [p for p in miss_files if p.endswith("_B.npz")]
        self.files = full_files + miss_files
        cleaned = []
        # remove any 0 value .npz files
        for f in self.files:
            try:
                # miss_sparse_mat = (sp.load_npz(path))
                # miss_numpy_array = np.array(miss_sparse_mat.toarray(), dtype = np.uint8)
                # return miss_numpy_array
                with np.load(f, allow_pickle=False) as npz:
                    k = "arr_0" if "arr_0" in npz.files else next(kk for kk in npz.files if isinstance(npz[kk], np.ndarray))
                    arr = np.array(npz[k])
                if arr.size == 0 or not np.isfinite(arr).any(): continue
                cleaned.append(f)
            except Exception:
                pass
        self.files = cleaned
        if not self.files:
            raise FileNotFoundError("No valid npz files found")
    def __len__(self): return len(self.files)
    def __getitem__(self, idx):
        # load files into npz
        with np.load(self.files[idx], allow_pickle=False) as npz:
            k = "arr_0" if "arr_0" in npz.files else next(kk for kk in npz.files if isinstance(npz[kk], np.ndarray))
            x = np.array(npz[k], dtype=np.float32)
        if x.ndim == 1: x = vector_to_square(x)
        elif x.ndim > 2: x = x[0]
        xmin, xmax = np.min(x), np.max(x)
        x = (x - xmin) / (xmax - xmin) if xmax > xmin else np.zeros_like(x, dtype=np.float32)
        x = torch.from_numpy(x)
        if x.ndim == 2: x = x.unsqueeze(0)
        elif x.ndim == 3 and x.shape[0] not in (1,3): x = x.permute(2,0,1)
        if x.shape[0] == 1: x = x.repeat(3,1,1)
        elif x.shape[0] > 3: x = x[:3]
        x = F.interpolate(x.unsqueeze(0), size=(self.image_size, self.image_size), mode="bilinear", align_corners=False).squeeze(0)
        x = x * 2 - 1
        return x

### MAKE SURE YOU ARE RUNNING ON GPU NOT CPU

In [10]:

print(torch.cuda.is_available())

True


In [11]:
!nvidia-smi


Wed Dec  3 10:32:43 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.261.03             Driver Version: 535.261.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:B3:00.0 Off |                  N/A |
| 44%   50C    P8              32W / 350W |     26MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0))

x = torch.rand(3, 3, device="cuda")
print(x)
print("x.device:", x.device)


CUDA available: True
Device: NVIDIA GeForce RTX 3090
tensor([[0.7799, 0.3640, 0.3493],
        [0.7623, 0.3210, 0.5388],
        [0.4987, 0.9869, 0.2862]], device='cuda:0')
x.device: cuda:0


In [13]:
torch.cuda.empty_cache()

In [14]:
# ds = HeatmapSmallDataset("/home/pranjali_jain/NCS/sparse-matrices/SPEC/SPEC189_2config/1-64-12", image_size=512)


In [15]:
# print(len(ds))

## Load Dataset & Configure Model Params

In [16]:
root = "/home/pranjali_jain/NCS/sparse-matrices/SPEC/SPEC189_2config/1-64-12/"
image_size = int(config.image_size) # 512x512
ds = HeatmapSmallDataset(root, image_size=image_size)

device = torch.device("cuda")
loader = DataLoader(ds, batch_size=1, shuffle=True, num_workers=4, pin_memory=True)


import time
it = iter(loader)
start = time.time()
for _ in range(200):
    batch = next(it)
end = time.time()
print("Pure dataloader sec/batch:", (end - start)/200)

batch = next(iter(loader))
print(len(ds), batch.shape)

model = model.to(device)
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

Pure dataloader sec/batch: 0.0064629590511322025
314017 torch.Size([1, 3, 512, 512])


## TRAINING LOOP

In [ ]:
# run = wandb.init(project="CacheBoxDModel", entity="dantle4564-uc-santa-barbara", name="CacheBoxTestRun")
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()

num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(loader, 1):
        batch = batch.to(device)
        t = torch.randint(0, noise_scheduler.config.num_train_timesteps, (batch.size(0),), device=device)
        noise = torch.randn_like(batch)
        noisy = noise_scheduler.add_noise(batch, noise, t)
        optimizer.zero_grad(set_to_none=True)
        with autocast():
            pred = model(noisy, t, return_dict=False)[0]
            loss = torch.nn.functional.l1_loss(pred, noise)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    # print(f"epoch {epoch+1}/{num_epochs} | loss {loss.item():.4f}")
    # wandb.log({"loss": loss.item()})
# run.finish()



<ipython-input-20-abc2e3738753>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


<ipython-input-20-abc2e3738753>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


# SAVE & LOAD MODEL AFTER TRAINING

In [ ]:

filename = 'cache_heatmap_diffusion_model_.sav' 

In [ ]:
joblib.dump(model, open(filename, 'wb')) 

In [ ]:
model = joblib.load(open(filename, 'rb'))

In [ ]:
import torchvision.transforms as T

wandb.init(project="CacheBoxDModel", entity="dantle4564-uc-santa-barbara", name="diffusion images")

scheduler = DDPMScheduler(num_train_timesteps=1000)

# generate only one image
num = 1
model.eval()
device = torch.device("cuda")

image_size = 512

with torch.no_grad():
    x = torch.randn(num, 3, image_size, image_size, device=device)
    scheduler.set_timesteps(scheduler.config.num_train_timesteps)
    for i,t in enumerate(scheduler.timesteps):
        with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
            model_input = scheduler.scale_model_input(x, t)
            noise_pred = model(model_input, t).sample
            x = scheduler.step(noise_pred, t, x).prev_sample
            if i % 10 == 0: 
                img = (x.clamp(-1, 1) + 1) / 2  # scale to [0,1]
                img_np = T.ToPILImage()(img[0].cpu())
                wandb.log({f"Time step {i}" : wandb.Image(img_np)})
        
    # save the single image
    img = (x.clamp(-1, 1) + 1) / 2
    print(img.type)
    print("Image shape:" , img.shape)
    arr0 = arr[0]                          # (3, H, W)

    # Option A: pixels as rows, channels as cols -> (H*W, 3)
    H, W = arr0.shape[1], arr0.shape[2]
    mat2d = arr0.reshape(3, H * W).T       # (H*W, 3)
    sparse_mat = sparse.csr_matrix(mat2d)
    sparse.save_npz("synthetic_heatmap.npz", sparse_mat)


In [ ]:
# paths
# python3 configwise_hitrate.py
# <real full heat map dir> = small_dataset/TEST/FULL

# SSIM

In [ ]:
from skimage.metrics import structural_similarity as ssim
data1 = sp.load_npz("/home/dantle/CacheBox/small_dataset/TEST/MISS/400.perlbench-41B/400.perlbench-50B.champsimtrace.xz.txt_0_B.npz")
data2 = sp.load_npz("/home/dantle/CacheBox/synthetic_heatmap.npz")

# print(data1.files)
# print(data2.files)
print(type(data1))
print(data1.shape)
print(data1.nnz)
img1 =  np.array(data1.toarray(), dtype = np.uint8)
# img2 = data2['arr_0']
# print("Shape 1:", img1.shape)
# print("Shape 2:", img2.shape)


#score, diff = ssim(img1, img2, full=True)

#print("SSIM:", score)



In [ ]:
import numpy as np
from scipy.sparse import load_npz
from skimage.metrics import structural_similarity as ssim
from skimage.transform import resize

# ---- 1. Load sparse test heatmap and make it dense ----
test_sp = load_npz("/home/dantle/CacheBox/small_dataset/TEST/MISS/400.perlbench-41B/400.perlbench-50B.champsimtrace.xz.txt_1_B.npz")    # your existing .npz
test_img = test_sp.toarray().astype(np.float32)  # shape: (512, 512)

# ---- 2. Load your synthetic dense heatmap ----
data = np.load("/home/dantle/CacheBox/synthetic_heatmap.npz")
# adjust key as needed:
synthetic = list(data.values())[0]               # or data["heatmap"]

# If synthetic came from (1, 3, 128, 128) torch tensor:
if synthetic.ndim == 4:
    # (1, 3, 128, 128) -> (3, 128, 128)
    synthetic = synthetic.squeeze(0)

if synthetic.ndim == 3:
    # collapse channels to grayscale (3, H, W) -> (H, W)
    synthetic = synthetic.mean(axis=0)

# Now synthetic is (H, W), probably (128, 128)

# ---- 3. Resize synthetic to match (512, 512) ----
synthetic_resized = resize(
    synthetic, 
    test_img.shape,       # (512, 512)
    anti_aliasing=True
).astype(np.float32)

print(test_img.shape)
# print(test_img)
print(synthetic_resized.shape)
# ---- 4. Compute SSIM ----
score, diff = ssim(test_img, synthetic_resized, full=True, data_range=1.0)
print("SSIM:", score)


In [ ]:
colormap = 'magma_r' #'PuOr'#
plt.style.use('seaborn-v0_8-deep')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = ['Helvetica']
Fontsize = 20
plt.rcParams['font.size'] = Fontsize

plt.rcParams['axes.facecolor'] = 'white'  
plt.rcParams['text.color'] = 'black'  
plt.rcParams['axes.labelcolor'] = 'black'  
plt.rcParams['xtick.color'] = 'black' 
plt.rcParams['ytick.color'] = 'black'  
plt.rcParams['axes.edgecolor'] = 'black'  
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['legend.handlelength'] = 0.7
plt.rcParams['legend.handleheight'] = 0.7
plt.rcParams['ytick.major.size'] = 4
plt.rcParams['ytick.major.width'] = 1

stepval = 256

figure, (ax1) = plt.subplots(1,1, figsize=(5, 5))
figure.patch.set_facecolor("white")


ax1.imshow(test_img, cmap=colormap)
# ax1.axis('off')
# ax1.set_title('Real full map')
ax1.set_xticks(ticks = np.arange(0, 513, step=stepval))
ax1.set_yticks(ticks = np.arange(0, 513, step=stepval))
plt.xticks(fontsize=Fontsize-2)
plt.yticks(fontsize=Fontsize-2)
plt.ylabel('cacheline address\nmodulo 512',labelpad=8)
plt.xlabel('Instructions',labelpad=8)
plt.tight_layout()
# plt.savefig("Graphs/heatmap_1x1_realfull", dpi=300, bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
colormap = 'magma_r' #'PuOr'#
plt.style.use('seaborn-v0_8-deep')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = ['Helvetica']
Fontsize = 20
plt.rcParams['font.size'] = Fontsize

plt.rcParams['axes.facecolor'] = 'white'  
plt.rcParams['text.color'] = 'black'  
plt.rcParams['axes.labelcolor'] = 'black'  
plt.rcParams['xtick.color'] = 'black' 
plt.rcParams['ytick.color'] = 'black'  
plt.rcParams['axes.edgecolor'] = 'black'  
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['legend.handlelength'] = 0.7
plt.rcParams['legend.handleheight'] = 0.7
plt.rcParams['ytick.major.size'] = 4
plt.rcParams['ytick.major.width'] = 1

stepval = 256

figure, (ax1) = plt.subplots(1,1, figsize=(5, 5))
figure.patch.set_facecolor("white")


ax1.imshow(synthetic_resized, cmap=colormap)
# ax1.axis('off')
# ax1.set_title('Real full map')
ax1.set_xticks(ticks = np.arange(0, 513, step=stepval))
ax1.set_yticks(ticks = np.arange(0, 513, step=stepval))
plt.xticks(fontsize=Fontsize-2)
plt.yticks(fontsize=Fontsize-2)
plt.ylabel('cacheline address\nmodulo 512',labelpad=8)
plt.xlabel('Instructions',labelpad=8)
plt.tight_layout()
# plt.savefig("Graphs/heatmap_1x1_realfull", dpi=300, bbox_inches='tight', pad_inches=0)
plt.show()